In [ ]:
from transformers import AutoImageProcessor, DetrForObjectDetection
import torch
from PIL import Image

image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")


In [ ]:
file = "road.png"
image = Image.open(file).convert("RGB")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
# convert outputs (bounding boxes and class logits) to Pascal VOC format (xmin, ymin, xmax, ymax)
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[
    0
]
print(results["scores"])
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

In [ ]:
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plot

image = read_image(file)
image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
image = image[:3, ...]
pred_boxes = results["boxes"].long()
labels = [f"{model.config.id2label[label.item()]}: {score} " for label, score in zip(results["labels"], results["scores"])]
output_image = draw_bounding_boxes(image, pred_boxes, labels, colors="red")

plot.imshow(output_image.permute(1, 2, 0))
plot.show()